In [1]:
from keras.utils import to_categorical
from keras.preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from PIL import Image
from sklearn.preprocessing import LabelEncoder

In [2]:
TRAIN_DIR = 'images/train'
TEST_DIR = 'images/validation'

In [3]:
def create_data_frame(dir):
    imagesPath = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir,label)):
            imagesPath.append(os.path.join(dir,label,imagename))
            labels.append(label)
        print(label," completed")
    return imagesPath,labels

In [4]:
train = pd.DataFrame()
train['image'],train['label'] = create_data_frame(TRAIN_DIR)

happy  completed
sad  completed
fear  completed
surprise  completed
neutral  completed
angry  completed
disgust  completed


In [5]:
test = pd.DataFrame()
test['image'],test['label'] = create_data_frame(TEST_DIR)

happy  completed
sad  completed
fear  completed
surprise  completed
neutral  completed
angry  completed
disgust  completed


In [6]:
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image, color_mode = 'grayscale')
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features), 48,48,1)
    return features

In [7]:
train_features = extract_features(train['image'])

  0%|          | 0/28821 [00:00<?, ?it/s]

In [8]:
test_features = extract_features(test['image'])

  0%|          | 0/7066 [00:00<?, ?it/s]

In [9]:
x_train = train_features/255.0
x_test = test_features/255.0

In [10]:
le = LabelEncoder()
le.fit(train['label'])

LabelEncoder()

In [11]:
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])

In [12]:
y_train = to_categorical(y_train, num_classes = 7)
y_test = to_categorical(y_test, num_classes = 7)

In [18]:
model = Sequential()

# convolutional Layers
model.add(Conv2D(128, kernel_size=(3,3) ,activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu')) 
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout (0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu')) 
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout (0.4))

model.add(Flatten())


# fully connected Layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))

#Output Layer
model.add(Dense(7,activation='softmax'))

/opt/anaconda3/envs/BabyCam/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [20]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics = ['accuracy'])

In [21]:
model.fit(x=x_train,y=y_train,batch_size = 128,epochs=100,validation_data=(x_test,y_test))

Epoch 1/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 214s 943ms/step - accuracy: 0.2282 - loss: 1.8455 - val_accuracy: 0.2583 - val_loss: 1.8102
Epoch 2/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 213s 943ms/step - accuracy: 0.2459 - loss: 1.8132 - val_accuracy: 0.2610 - val_loss: 1.7767
Epoch 3/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 190s 841ms/step - accuracy: 0.2647 - loss: 1.7697 - val_accuracy: 0.3431 - val_loss: 1.6370
Epoch 4/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 177s 785ms/step - accuracy: 0.3337 - loss: 1.6578 - val_accuracy: 0.4128 - val_loss: 1.5128
Epoch 5/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 176s 779ms/step - accuracy: 0.3966 - loss: 1.5456 - val_accuracy: 0.4616 - val_loss: 1.3989
Epoch 6/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 181s 802ms/step - accuracy: 0.4290 - loss: 1.4747 - val_accuracy: 0.4832 - val_loss: 1.3462
Epoch 7/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 183s 810ms/step - accuracy: 0.4503 - loss: 1.4206 - val_accuracy: 0.5123 - val_loss: 1.2937
Epoch 8/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 179s 791ms/step - accuracy: 

In [22]:
model.save("model_save.h5")

In [27]:
from keras.models import load_model
new_model = load_model("model_save.h5")

In [32]:
def ef(image):
    img = load_img(image, color_mode = 'grayscale')
    feature = np.array(img)
    feature = feature.reshape(1,48,48,1)
    return feature/255.0

In [35]:
image = 'images/train/angry/22.jpg'
img = ef(image)
pred = model.predict(img)
print(pred.argmax())

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
0
